# Data Preparation

In [1]:
import polars as pl

In [2]:
df = pl.read_csv("data/allseasonsgameinfo.csv", infer_schema_length=1000000)

In [3]:
print(
    "Season range:",
    df.select(pl.col("season").min()).item(),
    "-",
    df.select(pl.col("season").max()).item(),
)
df.head()

Season range: 1899 - 2024


gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season
str,str,str,str,i64,i64,str,str,i64,i64,bool,bool,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,i64
"""LS3189904140""","""CHN""","""LS3""","""LOU03""",18990414,0,"""0:00PM""","""day""",null,null,false,null,113,11500,"""unknown""","""unknown""","""unknown""","""0""","""unknown""","""-1""",null,null,null,"""burno101""","""warna901""",null,null,null,null,"""grifc101""","""cunnb103""",null,"""regular""",15,1,"""CHN""","""LS3""","""y""","""both""","""y""","""y""",null,1899
"""PHI189904140""","""WSN""","""PHI""","""PHI09""",18990414,0,"""0:00PM""","""day""",null,null,false,null,120,12000,"""unknown""","""unknown""","""unknown""","""0""","""unknown""","""-1""",null,null,null,"""huntj901""","""connt901""",null,null,null,null,"""piatw101""","""killf101""",null,"""regular""",5,6,"""PHI""","""WSN""","""y""","""both""","""y""","""y""",null,1899
"""BLN189904150""","""NY1""","""BLN""","""BAL07""",18990415,0,"""0:00PM""","""day""",null,null,false,null,130,3912,"""unknown""","""unknown""","""unknown""","""0""","""unknown""","""-1""",null,null,null,"""emslb101""","""bettw901""",null,null,null,null,"""kitsf101""","""dohee101""",null,"""regular""",3,5,"""BLN""","""NY1""","""y""","""both""","""y""","""y""",null,1899
"""BRO189904150""","""BSN""","""BRO""","""NYC12""",18990415,0,"""0:00PM""","""day""",null,null,false,null,120,20167,"""unknown""","""unknown""","""unknown""","""0""","""unknown""","""-1""",null,null,null,"""andre101""","""gaffj801""",null,null,null,null,"""nichk101""","""kennb101""",null,"""regular""",1,0,"""BSN""","""BRO""","""y""","""both""","""y""","""y""",null,1899
"""CIN189904150""","""PIT""","""CIN""","""CIN05""",18990415,0,"""0:00PM""","""day""",null,null,false,null,130,10000,"""unknown""","""unknown""","""unknown""","""0""","""unknown""","""-1""",null,null,null,"""sware101""","""warna901""",null,null,null,null,"""tannj101""","""hawlp101""",null,"""regular""",5,2,"""PIT""","""CIN""","""y""","""both""","""y""","""y""",null,1899


first we'll filter down to just seasons >= 1998, because we are going from the last time the league expanded

In [4]:
print(f"Before filtering we have {df.shape[0]} games in our dataset")
df = df.filter(pl.col("season") >= 1998)
print(f"After filtering we have {df.shape[0]} games in our dataset")

Before filtering we have 212555 games in our dataset
After filtering we have 65043 games in our dataset


In [5]:
# sanity check

print(df.select(pl.col("season").min()).item())
print(df.select(pl.col("season").max()).item())

1998
2024


In [6]:
df.describe()

statistic,gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season
str,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,str,str,str,str,f64
"""count""","""65043""","""65043""","""65043""","""65043""",65043.0,65043.0,"""62620""","""65043""",10834.0,10640.0,65043.0,44.0,65043.0,65043.0,"""65043""","""65043""","""65043""","""65043""","""65043""","""65043""","""50721""","""0""",56.0,"""65043""","""65043""","""65043""","""65043""","""11603""","""11603""","""65033""","""65033""","""32752""","""65043""",65043.0,65043.0,"""65033""","""65033""","""65043""","""65043""","""65043""","""65043""","""65043""",65043.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0,"""2423""","""0""",54209.0,54403.0,0.0,64999.0,0.0,0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""14322""","""65043""",64987.0,"""0""","""0""","""0""","""0""","""53440""","""53440""","""10""","""10""","""32291""","""0""",0.0,0.0,"""10""","""10""","""0""","""0""","""0""","""0""","""0""",0.0
"""mean""",null,null,null,null,2.0109e7,0.038328,null,null,8.957172,2.0,0.54567,1.0,178.42558,29192.342435,null,null,null,null,null,null,null,null,2.0148e7,null,null,null,null,null,null,null,null,null,null,4.517504,4.637394,null,null,null,null,null,null,null,2010.79901
"""std""",null,null,null,null,77574.451518,0.249825,null,null,0.289534,0.0,null,null,27.816959,11980.691816,null,null,null,null,null,null,null,null,73490.612282,null,null,null,null,null,null,null,null,null,null,3.210258,3.131412,null,null,null,null,null,null,null,7.75724
"""min""","""ALS199907130""","""ALS""","""ALS""","""ANA01""",1.9980331e7,0.0,"""01:20PM""","""day""",7.0,2.0,0.0,1.0,75.0,0.0,"""damp""","""drizzle""","""cloudy""","""0""","""fromcf""","""-1""",""" """,null,1.9980408e7,"""addir901""","""addir901""","""(none)""","""addir901""","""(none)""","""(none)""","""aardd001""","""aardd001""","""aardd001""","""allstar""",0.0,0.0,"""ALS""","""ALS""","""y""","""both""","""y""","""y""","""y""",1998.0
"""25%""",null,null,null,null,2.0040727e7,0.0,null,null,9.0,2.0,null,null,160.0,20232.0,null,null,null,null,null,null,null,null,2.0100417e7,null,null,null,null,null,null,null,null,null,null,2.0,2.0,null,null,null,null,null,null,null,2004.0
"""50%""",null,null,null,null,2.0110509e7,0.0,null,null,9.0,2.0,null,null,175.0,30020.0,null,null,null,null,null,null,null,null,2.0150912e7,null,null,null,null,null,null,null,null,null,null,4.0,4.0,null,null,null,null,null,null,null,2011.0
"""75%""",null,null,null,null,2.017083e7,0.0,null,null,9.0,2.0,null,null,193.0,38722.0,null,null,null,null,null,null,null,null,2.0210718e7,null,null,null,null,null,null,null,null,null,null,6.0,6.0,null,null,null,null,null,null,null,2017.0
"""max""","""WAS202409290""","""WAS""","""WAS""","""WIL02""",2.024103e7,2.0,"""9:59PM""","""night""",9.0,2.0,1.0,1.0,440.0,65975.0,"""wet""","""unknown""","""unknown""","""99""","""unknown""","""9""","""zibed701""",null,2.0240828e7,"""younl901""","""younl901""","""younl901""","""younl901""","""younl901""","""younl901""","""zycht001""","""zycht001""","""zycht001""","""worldseries""",30.0,29.0,"""WAS""","""WAS""","""y""","""both""","""y""","""y""","""y""",2024.0


create datetime column for easier manipulation of data by dates

In [7]:
df = df.with_columns(
    pl.col("date").cast(pl.String).str.strptime(pl.Date, format="%Y%m%d").alias("date")
)

In [8]:
df = df.sort(by=["date", "number"], descending=[False, False])
df.head()

gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season
str,str,str,str,date,i64,str,str,i64,i64,bool,bool,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,i64
"""ATL199803310""","""MIL""","""ATL""","""ATL02""",1998-03-31,0,"""4:11PM""","""day""",null,null,false,null,159,42891,"""dry""","""none""","""overcast""","""85""","""ltor""","""9""",null,null,null,"""froeb901""","""grege901""","""hirsm901""","""vanol901""",null,null,"""ligtk001""","""wickb001""",null,"""regular""",1,2,"""ATL""","""MIL""","""y""","""both""","""y""","""y""","""y""",1998
"""BAL199803310""","""KCA""","""BAL""","""BAL12""",1998-03-31,0,"""3:12PM""","""day""",null,null,true,null,158,46820,"""dry""","""none""","""sunny""","""91""","""ltor""","""13""",null,null,null,"""mckej901""","""clara901""","""joycj901""","""craft901""",null,null,"""belct001""","""mussm001""","""montj002""","""regular""",4,1,"""KCA""","""BAL""","""y""","""both""","""y""","""y""","""y""",1998
"""CIN199803310""","""SDN""","""CIN""","""CIN08""",1998-03-31,0,"""2:06PM""","""day""",null,null,false,null,168,54578,"""dry""","""none""","""cloudy""","""75""","""ltor""","""12""",null,null,null,"""marsr901""","""bonig901""","""hohnb901""","""herna901""",null,null,"""browk001""","""remlm001""",null,"""regular""",10,2,"""SDN""","""CIN""","""y""","""both""","""y""","""y""","""y""",1998
"""FLO199803310""","""CHN""","""FLO""","""MIA01""",1998-03-31,0,"""4:36PM""","""day""",null,null,false,null,188,41126,"""dry""","""none""","""cloudy""","""84""","""tocf""","""8""",null,null,null,"""pullf901""","""willc901""","""rapue901""","""drecb901""",null,null,"""hernl003""","""tapak001""",null,"""regular""",6,11,"""FLO""","""CHN""","""y""","""both""","""y""","""y""","""y""",1998
"""HOU199803310""","""SFN""","""HOU""","""HOU02""",1998-03-31,0,"""4:06PM""","""day""",null,null,false,null,269,43776,"""dry""","""none""","""dome""","""72""","""unknown""","""0""",null,null,null,"""tatat901""","""davig901""","""bellw901""","""mealj901""",null,null,"""johnj006""","""nitkc001""",null,"""regular""",9,4,"""SFN""","""HOU""","""y""","""both""","""y""","""y""","""y""",1998


In [9]:
for t in df.select(pl.col("gametype").unique()).to_series().to_list():
    print(t)

regular
worldseries
wildcard
playoff
divisionseries
allstar
lcs


In [10]:
df = df.filter(pl.col("gametype") == "regular")

what cols are we interested in and why?

In [11]:
df = df.select(
    [
        "date",
        "visteam",
        "hometeam",
        "vruns",
        "hruns",
    ]
)

In [12]:
print(df.select(pl.col("hometeam").n_unique()).item())

32


In [13]:
df.select(pl.col("hometeam").unique()).to_series().to_list()

['ARI',
 'TOR',
 'NYA',
 'FLO',
 'OAK',
 'TBA',
 'MIN',
 'SEA',
 'SLN',
 'CIN',
 'MIL',
 'KCA',
 'SDN',
 'WAS',
 'DET',
 'MIA',
 'BAL',
 'PHI',
 'NYN',
 'CHN',
 'CLE',
 'MON',
 'BOS',
 'HOU',
 'CHA',
 'COL',
 'TEX',
 'PIT',
 'ATL',
 'ANA',
 'SFN',
 'LAN']

In [14]:
team_name_corrections = {
    "NYA": "NYY",
    "CHA": "CHW",
    "CHN": "CHC",
    "LAN": "LAD",
    "KCA": "KCR",
    "NYM": "NYM",
    "SFN": "SFG",
    "SDN": "SDP",
    "SLN": "STL",
    "TBA": "TBR",
    "FLO": "MIA",  # team renamed
    "MON": "WAS",  # team moved
}

In [15]:
df = df.with_columns(
    [
        pl.col("hometeam").map_elements(
            lambda x: team_name_corrections.get(x, x), return_dtype=pl.String
        ),
        pl.col("visteam").map_elements(
            lambda x: team_name_corrections.get(x, x), return_dtype=pl.String
        ),
    ]
)

In [16]:
print(
    df.select(pl.col("hometeam").n_unique()).item(),
    df.select(pl.col("visteam").n_unique()).item(),
)

30 30


In [17]:
unique_vis_teams = df.select(pl.col("visteam").unique()).to_series().to_list()
unique_home_teams = df.select(pl.col("hometeam").unique()).to_series().to_list()
teams = set(unique_vis_teams) | set(unique_home_teams)

In [18]:
assert len(teams) == 30, f"Expected 30 teams, but found {len(teams)}"

In [19]:
df.write_parquet("data/prepared_game_data.parquet")